# Hermes Reply – Fase 5

**Contexto:** classificação binária do estado operacional (NORMAL vs ANOMALO) a partir de snapshots multivariados por máquina.

**Pipeline:**
1. Carregar `data/sensors_readings.csv`
2. Preparar features/target
3. Split estratificado (train/test)
4. Baseline + Logistic Regression + Random Forest
5. Métricas e gráficos (matriz de confusão, ROC, importância de variáveis)
6. Conclusão e próximos passos


In [ ]:
# Reprodutibilidade e imports
import numpy as np, random
np.random.seed(42); random.seed(42)

import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

DATA_PATH = '../data/sensors_readings.csv'
IMG_DIR = '../img'


In [ ]:
# Carregar dados
df = pd.read_csv(DATA_PATH)

print('Shape:', df.shape)
df.head()

In [ ]:
# Preparar X e y
X = df[['temperature_c','vibration_rms_g','pressure_bar','current_amp']].values
y = (df['label'] == 'ANOMALO').astype(int).values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print('Treino:', X_train.shape, 'Teste:', X_test.shape, 'Anomalias em teste:', y_test.sum())

In [ ]:
# Baseline
baseline = DummyClassifier(strategy='most_frequent')
baseline.fit(X_train, y_train)
b_acc = baseline.score(X_test, y_test)
print('Baseline accuracy:', round(b_acc, 4))

In [ ]:
# Logistic Regression
lr = Pipeline([('scaler', StandardScaler()),
               ('clf', LogisticRegression(max_iter=500, random_state=42))])
lr.fit(X_train, y_train)
lr_acc = lr.score(X_test, y_test)
lr_prob = lr.predict_proba(X_test)[:,1]
print('LR accuracy:', round(lr_acc, 4))

In [ ]:
# Random Forest
rf = RandomForestClassifier(n_estimators=300, class_weight='balanced', random_state=42)
rf.fit(X_train, y_train)
rf_acc = rf.score(X_test, y_test)
rf_prob = rf.predict_proba(X_test)[:,1]
print('RF accuracy:', round(rf_acc, 4))

In [ ]:
# Escolher melhor modelo (simplesmente pela acurácia)
best_name = 'RandomForest' if rf_acc >= lr_acc else 'LogisticRegression'
best_prob = rf_prob if best_name == 'RandomForest' else lr_prob
best_pred = (best_prob >= 0.5).astype(int)
print('Modelo escolhido:', best_name)

In [ ]:
# Matriz de confusão
cm = confusion_matrix(y_test, best_pred)

plt.figure(figsize=(5,4))
plt.imshow(cm, interpolation='nearest')
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
for (i,j), v in np.ndenumerate(cm):
    plt.text(j, i, str(v), ha='center', va='center')
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/confusion_matrix.png', dpi=150)
plt.show()

In [ ]:
# Curva ROC
fpr, tpr, _ = roc_curve(y_test, best_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(5,4))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.3f}')
plt.plot([0,1],[0,1], linestyle='--')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/roc_curve.png', dpi=150)
plt.show()

In [ ]:
# Importância de variáveis (se RF)
if best_name == 'RandomForest':
    importances = rf.feature_importances_
    plt.figure(figsize=(6,4))
    plt.bar(range(len(importances)), importances)
    plt.xticks(range(len(importances)), ['temperature_c','vibration_rms_g','pressure_bar','current_amp'], rotation=20)
    plt.title('Feature Importance (RandomForest)')
    plt.ylabel('Importance')
    plt.tight_layout()
    plt.savefig(f'{IMG_DIR}/feature_importance.png', dpi=150)
    plt.show()

## Conclusão breve
- Treinamos um classificador binário para detectar anomalias operacionais usando 4 variáveis (temperatura, vibração, pressão e corrente).
- Comparámos uma baseline, Regressão Logística e Random Forest, escolhendo o melhor por acurácia de teste.
- As métricas e gráficos (matriz de confusão, ROC e, se aplicável, importância de variáveis) documentam a qualidade do modelo.
- **Limitações:** dados simulados (não reais), possíveis vieses de geração e regra de rótulo simples. Em produção, recomenda-se validação por especialista, tuning e monitoramento.
